In [38]:
from PIL import Image
import os
import matplotlib.pyplot as plt

In [39]:
os.getcwd()

'/Users/sarora48/Documents/Courses/EVA/Assignment 16'

In [40]:
files = os.listdir("Photos/")
len(files), len([file for file in files if file.endswith(".jpg")])

(103, 99)

In [41]:
[file for file in files if not file.endswith(".jpg")]

['.DS_Store', 'images.jpeg', 'images (1).jpeg', '4.portraitist_cover.jpeg']

### Resizing the images

In [42]:
size = (400, 400)
for num in range(len(files)):
    file_address = os.path.join(os.getcwd(), f"Photos/{files[num]}")
    if not file_address.endswith("Store"):
        img = Image.open(file_address).resize((400, 400), Image.ANTIALIAS)
        if num < 10:
            img_name = f"./Resized/img_00{num}.jpg"
        elif ((num>9) and (num<100)):
            img_name = f"./Resized/img_0{num}.jpg"
        else:
            img_name = f"./Resized/img_{num}.jpg"
        img.save(img_name, "JPEG", optimize=True)

### Top-4 Anchor Boxes using K-means Clustering 

In [43]:
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [69]:
coordinates_df = pd.read_csv("Coordinates.csv")
coordinates_df.head()

,file_list,spatial_coordinates,metadata
0,"[""img_000.jpg""]","[2,159.214,154.791,48.648,67.322]",{}
1,"[""img_001.jpg""]","[2,29.975,46.192,63.882,153.808]",{}
2,"[""img_001.jpg""]","[2,159.214,51.106,93.366,191.646]",{}
3,"[""img_001.jpg""]","[2,292.383,26.536,65.357,164.619]",{}
4,"[""img_002.jpg""]","[2,190.172,49.631,117.445,239.312]",{}


In [70]:
coordinates_df['spatial_coordinates'] = coordinates_df.spatial_coordinates.str.strip("[]").str.split(",")
coordinates_df['file_list'] = coordinates_df.file_list.apply(lambda x: x.strip("[]"))
coordinates_df.head()

,file_list,spatial_coordinates,metadata
0,"""img_000.jpg""","[2, 159.214, 154.791, 48.648, 67.322]",{}
1,"""img_001.jpg""","[2, 29.975, 46.192, 63.882, 153.808]",{}
2,"""img_001.jpg""","[2, 159.214, 51.106, 93.366, 191.646]",{}
3,"""img_001.jpg""","[2, 292.383, 26.536, 65.357, 164.619]",{}
4,"""img_002.jpg""","[2, 190.172, 49.631, 117.445, 239.312]",{}


In [72]:
coordinates_df[['id','x', 'y', 'w', 'h']] = pd.DataFrame(coordinates_df.spatial_coordinates.values.tolist(), 
                                                  index=coordinates_df.index)
coordinates_df.drop(labels=['spatial_coordinates', 'id'], inplace=True, axis=1)
coordinates_df['file_list'] = coordinates_df.file_list.apply(lambda x: x.strip('""')) 
coordinates_df.head()

,file_list,metadata,x,y,w,h
0,img_000.jpg,{},159.214,154.791,48.648,67.322
1,img_001.jpg,{},29.975,46.192,63.882,153.808
2,img_001.jpg,{},159.214,51.106,93.366,191.646
3,img_001.jpg,{},292.383,26.536,65.357,164.619
4,img_002.jpg,{},190.172,49.631,117.445,239.312


In [89]:
metadata = coordinates_df.metadata.apply(lambda x: x.strip("{}").split(":"))
metadata[:10]

0            []
1            []
2            []
3            []
4            []
5    ["1", "8"]
6            []
7            []
8            []
9    ["1", "2"]
Name: metadata, dtype: object

In [99]:
coordinates_df['face_dir'] = pd.Series(metadata, index=coordinates_df.index)
coordinates_df.drop(labels=['metadata'], axis=1, inplace=True)
coordinates_df.head()

,file_list,x,y,w,h,face_dir
0,img_000.jpg,159.214,154.791,48.648,67.322,0
1,img_001.jpg,29.975,46.192,63.882,153.808,0
2,img_001.jpg,159.214,51.106,93.366,191.646,0
3,img_001.jpg,292.383,26.536,65.357,164.619,0
4,img_002.jpg,190.172,49.631,117.445,239.312,0


Face Direction Mapping:
- 0:"Front"
- 1:"Left"
- 2:"Right"
- 3:"Up"
- 4:"Down"
- 5:"UpLeft"
- 6:"UpRight"
- 7:"DownLeft"
- 8:"DownRight"	
- 9:"Top"
- 10:"Back"

In [102]:
X = np.asarray(coordinates_df[['w', 'h']])
kmeans = KMeans(n_clusters=4, random_state=42).fit(X)
kmeans.cluster_centers_

array([[199.30168421, 175.249     ],
       [107.45558621, 145.70868966],
       [156.93536364, 263.16745455],
       [ 47.08225   ,  51.4435625 ]])